In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import subprocess
from glob import glob

from pkg import u,m

# dataset_num = input("Enter dataset number: ")
# dataset = dataset_num.zfill(2) # string (ex '01')
dataset = '01'

pm = u.PathManager(dataset=dataset)
p = u.Processor(paths=pm, dataset=dataset)
clen, photon_energy = p.get_parameters()
print(f"clen: {clen}, photon_energy: {photon_energy}")
p = u.Processor(paths=pm, dataset=dataset)

u.get_counts(pm)

clen: 1.5, photon_energy: 6000
Paths refreshed for dataset 01.
Directory: peaks/01
	Total files: 40
	Normal images: 39
	Empty images: 1
Directory: labels/01
	Total files: 40
	Normal images: 39
	Empty images: 1
Directory: peaks_water_overlay/01
	Total files: 40
	Normal images: 39
	Empty images: 1


In [2]:
u.get_counts(pm)
%run process_directory.py ../../images  --percent_empty 0.5 
u.get_counts(pm)

Paths refreshed for dataset 01.
Directory: peaks/01
	Total files: 40
	Normal images: 39
	Empty images: 1
Directory: labels/01
	Total files: 40
	Normal images: 39
	Empty images: 1
Directory: peaks_water_overlay/01
	Total files: 40
	Normal images: 39
	Empty images: 1
Directories '01' through '09' already created in '../../images/labels'.
Directories '01' through '09' already created in '../../images/peaks'.
Directories '01' through '09' already created in '../../images/peaks_water_overlay'.
Directories '01' through '09' already created in '../../images/water'.
Directories for dataset 01 have been processed. Matching file counts found.
Directories for dataset 02 have been processed. Matching file counts found.
Directories for dataset 03 have been processed. Matching file counts found.
Directories for dataset 04 have been processed. Matching file counts found.
Directories for dataset 05 have been processed. Matching file counts found.
Directories for dataset 06 have been processed. Matchin

In [3]:
dm = u.DatasetManager(paths=pm, dataset=dataset, parameters=[clen,photon_energy], transform=None)

# peak, label, overlay are valid types
clen, photon_energy = p.get_parameters()

u.check_attributes(paths=pm, dataset=dataset, type='peak', clen=clen, photon_energy=photon_energy) 
u.check_attributes(paths=pm, dataset=dataset, type='overlay', clen=clen, photon_energy=photon_energy)
u.check_attributes(paths=pm, dataset=dataset, type='label', clen=clen, photon_energy=photon_energy)

train_loader, test_loader = u.prepare(data_manager=dm, batch_size=10)

Paths refreshed for dataset 01.
Directory: peaks/01
	Total files: 59
	Normal images: 39
	Empty images: 20
Directory: labels/01
	Total files: 59
	Normal images: 39
	Empty images: 20
Directory: peaks_water_overlay/01
	Total files: 59
	Normal images: 39
	Empty images: 20
Paths refreshed for dataset 01.
Actual percentage of empty images: 33.89830508474576% across peaks, water_overlays, and labels directories.



TypeError: retrieve_attributes requires at least 1 positional argument

In [5]:
model1 = m.BasicCNN1()
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model1.parameters(), lr=0.001)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cfg = {
    'num_epochs': 10,
    'num_classes': 2,
    'batch_size': train_loader.batch_size,
    'test_size': len(train_loader.dataset),
    'test_size': len(test_loader.dataset),
    'criterion': criterion,
    'optimizer': optimizer,
    'device': device,
    'model': model1,
    }

# arguments: self, model, loader: list, criterion, optimizer, device, cfg: dict
t = u.TrainTestModels(model=model1, loader=[train_loader, test_loader], criterion=criterion, optimizer=optimizer, device=device, cfg=cfg)
t.train()

KeyboardInterrupt: 

In [ ]:
test_peaks = pm.get_peak_image_paths()[0]
test_water = pm.get_water_image_paths()[0]
test_overlay = pm.get_peaks_water_overlay_image_paths()[0]
param1, param2 = (0.5, 9000), (0.5, 9000, True)

u.assign_attributes()